In [1]:
import json
from functools import partial
from typing import List

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from catboost import CatBoostClassifier, Pool
from catboost.utils import eval_metric
from scipy.spatial.distance import cosine, euclidean
from sklearn.metrics import pairwise_distances
from sklearn.model_selection import train_test_split

from sklearn.metrics import precision_score


### Load data

In [2]:
dataset = pd.read_parquet(r"C:\Users\druzh\Project_python\ozon_top_1\Datasets/train_pairs.parquet")
etl = pd.read_parquet(r"C:\Users\druzh\Project_python\ozon_top_1\Datasets/train_data.parquet")

Get raw data for each variantid.

In [3]:
features = (
    dataset
    .merge(
        etl
        .add_suffix('1'),
        on="variantid1"
    )
    .merge(
        etl
        .add_suffix('2'),
        on="variantid2"
    )
)

In [7]:
features.head(2)

,target,variantid1,variantid2,name1,categories1,color_parsed1,pic_embeddings_resnet_v11,main_pic_embeddings_resnet_v11,name_bert_641,characteristic_attributes_mapping1,name2,categories2,color_parsed2,pic_embeddings_resnet_v12,main_pic_embeddings_resnet_v12,name_bert_642,characteristic_attributes_mapping2
0,0.0,51197862,51198054,Удлинитель TDM Electric Люкс УЛ05В 5 м (SQ1303...,"{""1"": ""EPG"", ""2"": ""Электроника"", ""3"": ""Сетевые...",[белый],None,"[[-0.4304909, -0.49474272, -0.46439183, -0.060...","[-0.5104684, 0.56158644, 0.58873796, -0.529718...","{""Число жил"":[""3""],""Макс. нагрузка, Вт"":[""3500...",Удлинитель TDM Electric Люкс УЛ05В 1.5 м (SQ13...,"{""1"": ""EPG"", ""2"": ""Электроника"", ""3"": ""Сетевые...",[белый],None,"[[-0.42941108, -0.5129398, -0.4753536, -0.0677...","[-0.455473, 0.58157134, 0.5870387, -0.5325003,...","{""Электробезопасность"":[""Заземление""],""Длина к..."
1,0.0,51197862,51199884,Удлинитель TDM Electric Люкс УЛ05В 5 м (SQ1303...,"{""1"": ""EPG"", ""2"": ""Электроника"", ""3"": ""Сетевые...",[белый],None,"[[-0.4304909, -0.49474272, -0.46439183, -0.060...","[-0.5104684, 0.56158644, 0.58873796, -0.529718...","{""Число жил"":[""3""],""Макс. нагрузка, Вт"":[""3500...",Удлинитель TDM Electric Люкс УЛ05В 3 м (SQ1303...,"{""1"": ""EPG"", ""2"": ""Электроника"", ""3"": ""Сетевые...",[белый],None,"[[-0.43180764, -0.49580905, -0.5062628, -0.130...","[-0.5425725, 0.6415736, 0.51481575, -0.5687392...","{""Макс. нагрузка, Вт"":[""3500""],""Стандарт защит..."


In [8]:
feats = ["main_pic_embeddings_resnet_v11", "name_bert_641", "main_pic_embeddings_resnet_v12", "name_bert_642"]

In [9]:
X_train, X_test = train_test_split(
    features[feats + ["target", "variantid1", "variantid2"]], 
    test_size=0.1, random_state=42, stratify=features[["target"]])

y_test = X_test[["target"]]
X_test = X_test.drop(["target", "variantid1", "variantid2"], axis=1)

y_train = X_train["target"]
X_train = X_train.drop(["target","variantid1", "variantid2"], axis=1)

In [10]:
print(X_train.shape)

(275886, 4)


In [11]:
print(X_test.shape)

(30654, 4)


Features functions.

In [12]:
X_train_final = []

for i in range(len(X_train)):
    row = []
    row.extend(list(X_train[feats[0]].iloc[i])[0])
    row.extend(list(X_train[feats[1]].iloc[i]))
    row.extend(list(X_train[feats[2]].iloc[i])[0])
    row.extend(list(X_train[feats[3]].iloc[i]))
    X_train_final.append(row)

In [13]:
X_test_final = []

for i in range(len(X_test)):
    row = []
    row.extend(list(X_test[feats[0]].iloc[i])[0])
    row.extend(list(X_test[feats[1]].iloc[i]))
    row.extend(list(X_test[feats[2]].iloc[i])[0])
    row.extend(list(X_test[feats[3]].iloc[i]))
    X_test_final.append(row)

In [ ]:
from catboost import CatBoostClassifier
import optuna
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

def objective(trial):
    params = {
        'iterations': trial.suggest_int('iterations', 100, 1000),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.000001, 0.01),
        'depth': trial.suggest_int('depth', 2, 7),
        'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 0.1, 10.0),
        'border_count': trial.suggest_int('border_count', 5, 255),
        'random_seed': 42,
        'eval_metric': 'Recall',
        'loss_function': 'Logloss',
        'verbose': False
    }
    
    model = CatBoostClassifier(**params)
    model.fit(X_train_final, y_train, eval_set=(X_test_final, y_test), early_stopping_rounds=20, verbose=False)
    y_pred = model.predict(X_test_final)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

# Создание объекта study и запуск оптимизации
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# Получение лучших параметров
best_params = study.best_params
best_accuracy = study.best_value

print('Best parameters:', best_params)
print('Best accuracy:', best_accuracy)

[I 2023-05-29 04:11:56,738] A new study created in memory with name: no-name-5993fd61-738f-4895-b2bf-fcf9282f18c8
C:\Users\druzh\AppData\Local\Temp\ipykernel_6896\766522229.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.000001, 0.01),
C:\Users\druzh\AppData\Local\Temp\ipykernel_6896\766522229.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 0.1, 10.0),
[I 2023-05-29 04:12:39,415] Trial 0 finished with value: 0.6585437463300059 and parameters: {'iterations': 369, 'learning_rate': 0.009741282253431481, 'depth': 2, 'l2_leaf_

[I 2023-05-29 04:18:18,580] Trial 8 finished with value: 0.653030599595485 and parameters: {'iterations': 112, 'learning_rate': 0.0006015676101372897, 'depth': 3, 'l2_leaf_reg': 2.3405039327802646, 'border_count': 60}. Best is trial 7 with value: 0.6649376916552489.
C:\Users\druzh\AppData\Local\Temp\ipykernel_6896\766522229.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.000001, 0.01),
C:\Users\druzh\AppData\Local\Temp\ipykernel_6896\766522229.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 0.1, 10.0),
[I 2023-05-29 04:19

C:\Users\druzh\AppData\Local\Temp\ipykernel_6896\766522229.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.000001, 0.01),
C:\Users\druzh\AppData\Local\Temp\ipykernel_6896\766522229.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 0.1, 10.0),
[I 2023-05-29 04:24:44,560] Trial 17 finished with value: 0.6682325308279506 and parameters: {'iterations': 739, 'learning_rate': 0.00014066561803311245, 'depth': 7, 'l2_leaf_reg': 0.10164043550504681, 'border_count': 6}. Best is trial 15 with value: 0.6723103020812945.
C:\Users\druzh\

[I 2023-05-29 04:30:29,512] Trial 25 finished with value: 0.6551836628172506 and parameters: {'iterations': 924, 'learning_rate': 0.00021632677107611138, 'depth': 4, 'l2_leaf_reg': 0.14614148976948327, 'border_count': 113}. Best is trial 15 with value: 0.6723103020812945.
C:\Users\druzh\AppData\Local\Temp\ipykernel_6896\766522229.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.000001, 0.01),
C:\Users\druzh\AppData\Local\Temp\ipykernel_6896\766522229.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 0.1, 10.0),
[I 2023-05-29

In [20]:
model = CatBoostClassifier(**best_params)

model.fit(X_train_final, y_train)

0:	learn: 0.6931260	total: 77.9ms	remaining: 59s
1:	learn: 0.6931250	total: 157ms	remaining: 59.3s
2:	learn: 0.6931241	total: 232ms	remaining: 58.4s
3:	learn: 0.6931237	total: 313ms	remaining: 59s
4:	learn: 0.6931234	total: 402ms	remaining: 1m
5:	learn: 0.6931232	total: 498ms	remaining: 1m 2s
6:	learn: 0.6931235	total: 590ms	remaining: 1m 3s
7:	learn: 0.6931234	total: 679ms	remaining: 1m 3s
8:	learn: 0.6931234	total: 769ms	remaining: 1m 4s
9:	learn: 0.6931234	total: 855ms	remaining: 1m 3s
10:	learn: 0.6931234	total: 944ms	remaining: 1m 4s
11:	learn: 0.6931231	total: 1.03s	remaining: 1m 4s
12:	learn: 0.6931226	total: 1.12s	remaining: 1m 4s
13:	learn: 0.6931223	total: 1.2s	remaining: 1m 3s
14:	learn: 0.6931222	total: 1.29s	remaining: 1m 3s
15:	learn: 0.6931219	total: 1.37s	remaining: 1m 3s
16:	learn: 0.6931216	total: 1.46s	remaining: 1m 3s
17:	learn: 0.6931214	total: 1.54s	remaining: 1m 3s
18:	learn: 0.6931200	total: 1.62s	remaining: 1m 3s
19:	learn: 0.6931188	total: 1.72s	remaining: 1m 

164:	learn: 0.6929429	total: 14.7s	remaining: 52.7s
165:	learn: 0.6929411	total: 14.7s	remaining: 52.6s
166:	learn: 0.6929399	total: 14.8s	remaining: 52.5s
167:	learn: 0.6929390	total: 14.9s	remaining: 52.4s
168:	learn: 0.6929380	total: 15s	remaining: 52.3s
169:	learn: 0.6929367	total: 15.1s	remaining: 52.3s
170:	learn: 0.6929357	total: 15.2s	remaining: 52.2s
171:	learn: 0.6929346	total: 15.3s	remaining: 52.1s
172:	learn: 0.6929335	total: 15.4s	remaining: 52s
173:	learn: 0.6929321	total: 15.5s	remaining: 52s
174:	learn: 0.6929307	total: 15.6s	remaining: 51.8s
175:	learn: 0.6929298	total: 15.6s	remaining: 51.7s
176:	learn: 0.6929284	total: 15.7s	remaining: 51.7s
177:	learn: 0.6929270	total: 15.8s	remaining: 51.6s
178:	learn: 0.6929257	total: 15.9s	remaining: 51.5s
179:	learn: 0.6929244	total: 16s	remaining: 51.4s
180:	learn: 0.6929232	total: 16.1s	remaining: 51.3s
181:	learn: 0.6929218	total: 16.2s	remaining: 51.2s
182:	learn: 0.6929204	total: 16.3s	remaining: 51.1s
183:	learn: 0.692918

323:	learn: 0.6927465	total: 29.4s	remaining: 39.4s
324:	learn: 0.6927453	total: 29.5s	remaining: 39.3s
325:	learn: 0.6927442	total: 29.6s	remaining: 39.3s
326:	learn: 0.6927429	total: 29.7s	remaining: 39.2s
327:	learn: 0.6927418	total: 29.8s	remaining: 39.1s
328:	learn: 0.6927408	total: 29.9s	remaining: 39s
329:	learn: 0.6927396	total: 30s	remaining: 38.9s
330:	learn: 0.6927386	total: 30.1s	remaining: 38.8s
331:	learn: 0.6927375	total: 30.2s	remaining: 38.7s
332:	learn: 0.6927360	total: 30.3s	remaining: 38.7s
333:	learn: 0.6927351	total: 30.4s	remaining: 38.6s
334:	learn: 0.6927336	total: 30.5s	remaining: 38.5s
335:	learn: 0.6927326	total: 30.6s	remaining: 38.4s
336:	learn: 0.6927312	total: 30.7s	remaining: 38.3s
337:	learn: 0.6927301	total: 30.8s	remaining: 38.2s
338:	learn: 0.6927290	total: 30.9s	remaining: 38.1s
339:	learn: 0.6927276	total: 31s	remaining: 38.1s
340:	learn: 0.6927266	total: 31.1s	remaining: 38s
341:	learn: 0.6927248	total: 31.1s	remaining: 37.9s
342:	learn: 0.692723

482:	learn: 0.6925504	total: 44.5s	remaining: 25.3s
483:	learn: 0.6925493	total: 44.6s	remaining: 25.2s
484:	learn: 0.6925477	total: 44.7s	remaining: 25.1s
485:	learn: 0.6925466	total: 44.8s	remaining: 25s
486:	learn: 0.6925455	total: 44.8s	remaining: 25s
487:	learn: 0.6925443	total: 44.9s	remaining: 24.9s
488:	learn: 0.6925433	total: 45s	remaining: 24.8s
489:	learn: 0.6925422	total: 45.1s	remaining: 24.7s
490:	learn: 0.6925411	total: 45.2s	remaining: 24.6s
491:	learn: 0.6925397	total: 45.3s	remaining: 24.5s
492:	learn: 0.6925383	total: 45.4s	remaining: 24.4s
493:	learn: 0.6925372	total: 45.5s	remaining: 24.3s
494:	learn: 0.6925360	total: 45.6s	remaining: 24.2s
495:	learn: 0.6925348	total: 45.7s	remaining: 24.1s
496:	learn: 0.6925337	total: 45.8s	remaining: 24.1s
497:	learn: 0.6925326	total: 45.9s	remaining: 24s
498:	learn: 0.6925314	total: 46s	remaining: 23.9s
499:	learn: 0.6925302	total: 46.1s	remaining: 23.8s
500:	learn: 0.6925291	total: 46.2s	remaining: 23.7s
501:	learn: 0.6925278	

643:	learn: 0.6923525	total: 59.6s	remaining: 10.6s
644:	learn: 0.6923512	total: 59.7s	remaining: 10.5s
645:	learn: 0.6923500	total: 59.8s	remaining: 10.4s
646:	learn: 0.6923486	total: 59.9s	remaining: 10.3s
647:	learn: 0.6923477	total: 60s	remaining: 10.2s
648:	learn: 0.6923465	total: 1m	remaining: 10.1s
649:	learn: 0.6923453	total: 1m	remaining: 10s
650:	learn: 0.6923442	total: 1m	remaining: 9.91s
651:	learn: 0.6923426	total: 1m	remaining: 9.81s
652:	learn: 0.6923416	total: 1m	remaining: 9.72s
653:	learn: 0.6923406	total: 1m	remaining: 9.63s
654:	learn: 0.6923395	total: 1m	remaining: 9.54s
655:	learn: 0.6923383	total: 1m	remaining: 9.44s
656:	learn: 0.6923373	total: 1m	remaining: 9.35s
657:	learn: 0.6923362	total: 1m	remaining: 9.26s
658:	learn: 0.6923348	total: 1m 1s	remaining: 9.17s
659:	learn: 0.6923333	total: 1m 1s	remaining: 9.07s
660:	learn: 0.6923324	total: 1m 1s	remaining: 8.98s
661:	learn: 0.6923311	total: 1m 1s	remaining: 8.89s
662:	learn: 0.6923298	total: 1m 1s	remaining: 

In [21]:
import random

from sklearn.metrics import mean_squared_error
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score
prediction = model.predict(X_test_final)

mse = mean_squared_error(y_test, prediction)
print("Mean Squared Error:", mse)

f1 = f1_score(y_test, prediction)
print("f1:", f1)

accuracy = accuracy_score(y_test, prediction)
precision = precision_score(y_test, prediction)
recall = recall_score(y_test, prediction)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)

Mean Squared Error: 0.3251777908266458
f1: 0.5445074026686163
Accuracy: 0.6748222091733542
Precision: 0.7107240844566385
Recall: 0.44130064439671135


In [36]:
model.save_model("catboost_with_embs_only_optune_max.cbm")